In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('clust_proj').getOrCreate()

In [3]:
df = spark.read.csv('/FileStore/tables/hack_data.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)

In [5]:
df.head(2)

Out[ 5 ]: 
[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08)]

In [6]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [7]:
df.columns

Out[ 7 ]: 
['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [8]:
vec_assem = VectorAssembler(inputCols=['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 
                                         'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed'],
                            outputCol='features')

In [9]:
final_df = vec_assem.transform(df)

In [10]:
final_df.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

In [11]:
from pyspark.ml.feature import StandardScaler

In [12]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

In [13]:
scaled_final_df = scaler.fit(final_df).transform(final_df)

In [14]:
kmeans2 = KMeans(featuresCol='features_scaled', k=2)
kmeans3 = KMeans(featuresCol='features_scaled', k=3)

In [15]:
model2 = kmeans2.fit(scaled_final_df)
model3 = kmeans3.fit(scaled_final_df)

In [16]:
model3.transform(scaled_final_df).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 2| 84|
 0| 83|
+----------+-----+

In [17]:
model2.transform(scaled_final_df).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+